Poniżej są siatki parametrów i  ich wyniki z kaggla w postaci wykresów coyb dobrze widać było

In [19]:
from plotly.graph_objects import Figure, Scatter3d

def scatter_3d(df, 
               x_col, y_col, z_col, size_col, color_col, 
               x_title, y_title, z_title, size_title, color_title, 
               color_scale=[0,1], scatter=0.01):

    from numpy import random, log, mean

    fig = Figure(data=[Scatter3d(
        x=df[x_col] + mean(df[x_col]) * random.normal(0, scatter, size=len(df)),
        y=df[y_col] + mean(df[y_col]) * random.normal(0, scatter, size=len(df)),
        z=df[z_col] + mean(df[z_col]) * random.normal(0, scatter, size=len(df)),
        mode='markers',
        marker=dict(
            size=log(df[size_col])*2+1,
            sizemode='diameter',
            opacity=0.8,
            color=df[color_col],
            colorscale='thermal',
            colorbar=dict(title=color_title),
            cmin=color_scale[0] if color_scale else None,
            cmax=color_scale[1] if color_scale else None
        ),
        text=
            x_title + '=' + df[x_col].astype(str) + '; ' + 
            y_title + '=' + df[y_col].astype(str) + '; ' + 
            z_title + '=' + df[z_col].astype(str) + '; ' +
            size_title + '='+ df[size_col].astype(str) + '; ' + 
            color_title + '='+ round(df[color_col],3).astype(str),
        hoverinfo='text'
    )])

    fig.update_layout(
        title="Parametry i wyniki walidacji krzyżowej",
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title=x_title,
            yaxis_title=y_title,
            zaxis_title=z_title
        )
    )
    
    return fig

def scatter_3d_params(df, scatter=0.01, color_scale=[0,1]):

    return scatter_3d(df, 
        'min_samples_leaf', 'min_samples_split', 'ccp_alpha', 'n_estimators', 'mean_test_score', 
        'Min.L', 'Min.S', 'A', 'ne', 'score', scatter = 0.01, color_scale=color_scale
    )

In [14]:
import glob
import pandas as pd

dfs = [pd.read_csv(file_path, index_col=0) for file_path in glob.glob('output@kaggle/*.csv')]
df = pd.concat(dfs, ignore_index=True); del dfs
df.columns = [col.replace('param_', '') if col.startswith('param_') else col for col in df.columns]
df = df.sort_values(by='mean_test_score', ascending=True)
df

,ccp_alpha,max_samples,min_samples_leaf,min_samples_split,n_estimators,mean_test_score,std_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,rank_test_score
999,0.15,0.01,1000,80,20,0.079953,0.058024,0.383956,0.008042,0.116328,0.003498,0.154023,0.073502,0.012334,500
998,0.10,0.01,1000,20,20,0.080620,0.058061,0.390284,0.003302,0.118421,0.001184,0.154269,0.075237,0.012356,499
997,0.30,0.01,1000,60,20,0.080740,0.057997,0.390258,0.005974,0.115654,0.002427,0.154184,0.075638,0.012397,498
996,0.00,0.01,1000,20,20,0.080769,0.057507,0.387145,0.003912,0.116838,0.001643,0.153105,0.076791,0.012412,497
995,0.00,0.01,1000,80,20,0.080821,0.058047,0.387264,0.003151,0.118110,0.000954,0.154323,0.075728,0.012412,496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,0.15,0.01,20,20,400,0.192681,0.139500,11.197892,0.017041,7.903418,0.067293,0.370014,0.178879,0.029148,5
2003,0.30,0.01,20,40,200,0.192748,0.139801,5.576735,0.070205,3.918408,0.011503,0.370570,0.178678,0.028997,4
2002,0.15,0.01,20,40,1000,0.192800,0.140041,27.847759,0.051734,19.610601,0.067325,0.371244,0.177980,0.029176,3
2001,0.30,0.01,20,20,600,0.192905,0.140287,16.394314,0.054530,11.813535,0.016825,0.371717,0.177930,0.029066,2


In [20]:
scatter_3d_params(df, color_scale=None)

In [35]:
select = (df['ccp_alpha']==0) & (df['min_samples_leaf'] <= 100) & (df['min_samples_split'] <= 100)

scatter_3d_params(df[select], color_scale=None)